# Walkthrough 

This notebook will provide examples on how to use the .py file functions and classes in this repository. 

## How to use Solvents.py

In [1]:
from Solvents import SolventData

In [2]:
dielectric_path = "data archive/dielectric.csv"
solvent_data = SolventData(dielectric_path)
odichlorbenzene_dict = solvent_data.getSolventData("1,2-Dichlorobenzene")
print(odichlorbenzene_dict)

Dielectric constant for 1,2-Dichlorobenzene found in University of Washington's dielectric chart.
{'Solvent': '1,2-Dichlorobenzene', 'Solvent_SMILES': 'Clc1ccccc1Cl', 'Solvent_SMILES_2': nan, 'SOLV_PARAM_s_g': 0.90799, 'SOLV_PARAM_b_g': 0.86727, 'SOLV_PARAM_e_g': 0.12499, 'SOLV_PARAM_l_g': 1.12073, 'SOLV_PARAM_a_g': 0.42881, 'SOLV_PARAM_c_g': -0.85004, 'SOLV_PARAM_abraham err': 0.07, 'SOLV_PARAM_s_h': -9.81485, 'SOLV_PARAM_b_h': -5.48747, 'SOLV_PARAM_e_h': 4.46035, 'SOLV_PARAM_l_h': -9.14429, 'SOLV_PARAM_a_h': -8.69333, 'SOLV_PARAM_c_h': -4.66197, 'SOLV_PARAM_mintz err': 0.31, 'SOLV_PARAM_visc at 298 K (cP)': 1.3272, 'SOLV_PARAM_dielectric constant': 9.93}


## How to use Featurize.py

In [1]:
import Featurize

Say you want experimental data on a solvent, dioxane: 

In [7]:
solvent_name = "dioxane"
experimental_solvent = Featurize.getExperimentalSolvent(solvent_name)
print(experimental_solvent)

# Get a specific value in the dictionary:
solvent_smiles = experimental_solvent["Solvent_SMILES"]
print(f"solvent smiles: {solvent_smiles}")

{'Solvent': 'dioxane', 'Solvent_SMILES': 'C1COCCO1', 'Solvent_SMILES_2': nan, 'SOLV_PARAM_s_g': 1.79, 'SOLV_PARAM_b_g': 0.02, 'SOLV_PARAM_e_g': -0.37, 'SOLV_PARAM_l_g': 0.91, 'SOLV_PARAM_a_g': 2.98, 'SOLV_PARAM_c_g': -0.02, 'SOLV_PARAM_visc at 298 K (cP)': 1.18, 'SOLV_PARAM_dielectric constant': 2.25}
solvent smiles: C1COCCO1


A few examples on how you would use individual feature functions:

In [18]:
# Example monomer: L-Lactide
# define parameters that will be passed into functions
canonical_monomer_smiles = "C[C@@H]1OC(=O)[C@H](C)OC1=O"
dp = 5
solvent_smiles = "C1COCCO1"
monomer_base_state = "s"
polymerization_type = "ROP"

In [14]:
rdkit_features = Featurize.getRdkitDescriptors(canonical_monomer_smiles)
print(f"RDKIT features: {rdkit_features}")
pep = Featurize.getPEP(canonical_monomer_smiles, polymerization_type,dp)
print(f"Enthalpy PEP Feature: {pep}")
solvent_features = Featurize.getSolventFeatures(canonical_monomer_smiles, monomer_base_state, solvent_smiles)
print(f"Solvent features: {solvent_features}")

RDKIT features: {'RDKIT_MaxAbsEStateIndex': 10.650601851851853, 'RDKIT_MaxEStateIndex': 10.650601851851853, 'RDKIT_MinAbsEStateIndex': 0.47953703703703643, 'RDKIT_MinEStateIndex': -0.7465740740740745, 'RDKIT_qed': 0.44398578954674006, 'RDKIT_SPS': 33.0, 'RDKIT_MolWt': 144.12599999999998, 'RDKIT_HeavyAtomMolWt': 136.06199999999998, 'RDKIT_ExactMolWt': 144.042258736, 'RDKIT_NumValenceElectrons': 56, 'RDKIT_NumRadicalElectrons': 0, 'RDKIT_MaxPartialCharge': 0.3474957553971189, 'RDKIT_MinPartialCharge': -0.4479149703022866, 'RDKIT_MaxAbsPartialCharge': 0.4479149703022866, 'RDKIT_MinAbsPartialCharge': 0.3474957553971189, 'RDKIT_FpDensityMorgan1': 1.0, 'RDKIT_FpDensityMorgan2': 1.3, 'RDKIT_FpDensityMorgan3': 1.6, 'RDKIT_BCUT2D_MWHI': 16.64481001536744, 'RDKIT_BCUT2D_MWLOW': 10.29938649161736, 'RDKIT_BCUT2D_CHGHI': 2.315265141544972, 'RDKIT_BCUT2D_CHGLO': -2.219035522108289, 'RDKIT_BCUT2D_LOGPHI': 2.079846784377933, 'RDKIT_BCUT2D_LOGPLOW': -2.404118729607668, 'RDKIT_BCUT2D_MRHI': 5.8632526607

## Create a .csv with all feature data given you have a .csv with monomer strings

In [1]:
import pandas as pd
import numpy as np
import os
import Featurize

In [2]:
# path to .csv file will monomer data that you want to featurize
infile_path = "monomer data build/entropy.csv"

filename = os.path.splitext(os.path.basename(infile_path))[0]
directory_path = os.path.dirname(infile_path)

new_file_name = "featurized_" +  filename + ".csv"
featurized_path = directory_path + "/" + new_file_name
print(featurized_path)

# dH or dS
target = "dS"

monomer data build/featurized_entropy.csv


In [3]:
# If getAllFeatures doesn't seem to be working, make sure that your pandas df is indexed. If it is not uncomment the below code: 

# import pandas as pd

# infile_path = "data archive/dielectric.csv"
# # Read the CSV file
# df = pd.read_csv(infile_path)

# # Add an index column
# df_reset = df.reset_index()

# # Save the DataFrame to a new CSV file
# df_reset.to_csv(infile_path, index=False)

In [4]:
# choose a degree of polymerization
dp = 5

# turn .csv into a pandas dataframe
unfeaturized_df = pd.read_csv(infile_path, index_col=0, encoding="utf-8")

featurized_df = np.nan

for index,row in unfeaturized_df.iterrows():
    #  get the parameter data
    canonical_monomer_smiles = row["Canonical SMILES"]
    print(f"Featurizing {canonical_monomer_smiles}")

    #  get the monomer and polymer base states
    base_state = row["BASE_State"]
    monomer_base_state_col = base_state[0]
    poly_base_state_col = base_state[1:]

    # get the parameters that will go into getAllFeatures
    solvent_name = row["Solvent"]
    # print(solvent_name)
    monomer_base_state = monomer_base_state_col
    polymerization_type = row["BASE_Category"]

    update_dict = Featurize.getAllFeatures(canonical_monomer_smiles, monomer_base_state, polymerization_type, dp, solvent_name, target)
    if target == "dH":
        update_dict["dH (KJ/mol)"] = row["dH (KJ/mol)"]
    else:
        update_dict["dS (J/mol/K)"] = row["dS (J/mol/K)"]

    update_dict["BASE_Polymer_State"] = poly_base_state_col

    if index ==0:
        featurized_df = pd.DataFrame([update_dict])
        featurized_df.to_csv(featurized_path, encoding='utf-8-sig')

    # Update the DataFrame containing featurized data
    featurized_df.loc[len(featurized_df)] = update_dict

    # Update the csv by overwriting latest version
    featurized_df.to_csv(featurized_path, encoding='utf-8-sig')

print("OPERATION COMPLETE :)")

Featurizing CCC1(CC)CCCOC1=O


[13:41:00] DEPRECATION WARNING: please use MorganGenerator
[13:41:00] DEPRECATION WARNING: please use MorganGenerator
1 molecule converted
1 molecule converted
1 molecule converted


Featurizing CCCC1(CCC)CCCOC1=O


1 molecule converted
1 molecule converted
1 molecule converted


Featurizing CCCCC1(CCCC)CCCOC1=O


...

1 molecule converted
1 molecule converted
1 molecule converted


Featurizing C[C@@H]1OC(=O)[C@H](C)OC1=O


1 molecule converted
1 molecule converted


OPERATION COMPLETE :)


1 molecule converted


## How to use Polymerize.py

In [ ]:
from Polymerize import Polymerization
import numpy as np
import pandas as pd

In [ ]:
canonical_monomer_smiles = "C[C@@H]1OC(=O)[C@H](C)OC1=O"
polymerization_type = "ROP"
dp = 5

In [ ]:
# add the polymers and solvents to data
polymerize_data = Polymerization(canonical_monomer_smiles, polymerization_type,dp)
print(polymerize_data.main())

{'DP_0': 'C[C@@H]1OC(=O)[C@H](C)OC1=O', 'DP_1': 'C[C@H](O)C(=O)O[C@@H](C)C(=O)C', 'DP_2': 'C[C@H](O)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)C', 'DP_3': 'C[C@H](O)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)C', 'DP_4': 'C[C@H](O)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)C', 'DP_5': 'C[C@H](O)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)O[C@@H](C)C(=O)C'}


## How to use Impute.py

In [5]:
from Impute import ImputeData

In [6]:
# path to .csv file with monomer feature data
infile_path = "monomer data build/featurized_entropy.csv"
target = "dS"

In [7]:
impute_data = ImputeData(infile_path, target)
impute_data.main()

DIPOLE_gasteiger
DIPOLE_mmff94
DIPOLE_eem2015bm
SOLUTE_PARAM_E_GC
SOLUTE_PARAM_S_GC
SOLUTE_PARAM_A_GC
SOLUTE_PARAM_B_GC
SOLUTE_PARAM_L_GC
SOLUTE_PARAM_V_GC
SOLUTE_PARAM_E_ML
SOLUTE_PARAM_S_ML
SOLUTE_PARAM_A_ML
SOLUTE_PARAM_B_ML
SOLUTE_PARAM_L_ML
SOLUTE_PARAM_V_ML
Imputing Complete
One-hot Encoding Complete
Columns removed: ['RDKIT_fr_Al_COO', 'RDKIT_fr_Al_OH', 'RDKIT_fr_Al_OH_noTert', 'RDKIT_fr_ArN', 'RDKIT_fr_Ar_COO', 'RDKIT_fr_Ar_N', 'RDKIT_fr_Ar_NH', 'RDKIT_fr_Ar_OH', 'RDKIT_fr_COO', 'RDKIT_fr_COO2', 'RDKIT_fr_C_S', 'RDKIT_fr_HOCCN', 'RDKIT_fr_Imine', 'RDKIT_fr_NH0', 'RDKIT_fr_NH1', 'RDKIT_fr_NH2', 'RDKIT_fr_N_O', 'RDKIT_fr_Ndealkylation1', 'RDKIT_fr_Ndealkylation2', 'RDKIT_fr_Nhpyrrole', 'RDKIT_fr_SH', 'RDKIT_fr_alkyl_carbamate', 'RDKIT_fr_alkyl_halide', 'RDKIT_fr_amide', 'RDKIT_fr_amidine', 'RDKIT_fr_aniline', 'RDKIT_fr_aryl_methyl', 'RDKIT_fr_azide', 'RDKIT_fr_azo', 'RDKIT_fr_barbitur', 'RDKIT_fr_benzene', 'RDKIT_fr_benzodiazepine', 'RDKIT_fr_bicyclic', 'RDKIT_fr_diazo', 'RDKIT_f

## How to use Models.py

In [1]:
import Models

In [2]:
infile_path = "monomer data build/encoded_imputed_featurized_entropy.csv"
n_iters = 100
target = "dS (J/mol/K)"

Models.main(infile_path, n_iters, target)

UNIQUE DATASETS CREATED:
    1: Full + solvent parameters
    2: Full without solvent parameters
    3: Solvents only (with solvent parameters)
    4: Bulk only
Dataset 1 Hyperparameters Complete


/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singul

Dataset 2 Hyperparameters complete


/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=2.4832e-53): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=1.39797e-55): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=1.39428e-55): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=1.39789e-55): result may not be accurate.
  dual_coef =

Dataset 3 Hyperparameters complete


/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/Users/hunter/anaconda3/envs/poly_broadbelt/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singul

Dataset 4 Hyperparameters complete
MODELS FOR DATASET 1 CREATED
MODELS FOR DATASET 2 CREATED
MODELS FOR DATASET 3 CREATED
MODELS FOR DATASET 4 CREATED
RUNNING MODELS FOR DATASET 1
RUNNING XGB
TRAIN XGB LOOCV COMPLETE
TRAIN XGB COMPLETE
RUNNING MODELS FOR DATASET 2
RUNNING XGB
TRAIN XGB LOOCV COMPLETE
TRAIN XGB COMPLETE
RUNNING SVR
TRAIN SVR LOOCV COMPLETE
TRAIN SVR COMPLETE
RUNNING GP
TRAIN GP LOOCV COMPLETE
TRAIN GP COMPLETE
RUNNING KR
TRAIN KR LOOCV COMPLETE
TRAIN KR COMPLETE
RUNNING RF
TRAIN RF LOOCV COMPLETE
TRAIN RF COMPLETE
RUNNING MODELS FOR DATASET 3
RUNNING XGB
TRAIN XGB LOOCV COMPLETE
TRAIN XGB COMPLETE
RUNNING SVR
TRAIN SVR LOOCV COMPLETE
TRAIN SVR COMPLETE
RUNNING GP
TRAIN GP LOOCV COMPLETE
TRAIN GP COMPLETE
RUNNING KR
TRAIN KR LOOCV COMPLETE
TRAIN KR COMPLETE
RUNNING RF
TRAIN RF LOOCV COMPLETE
TRAIN RF COMPLETE
RUNNING MODELS FOR DATASET 4
RUNNING XGB
TRAIN XGB LOOCV COMPLETE
TRAIN XGB COMPLETE
RUNNING SVR
TRAIN SVR LOOCV COMPLETE
TRAIN SVR COMPLETE
RUNNING GP
TRAIN GP LOO

In [7]:
from rdkit import Chem

canonical_monomer_smiles = "C1=CC=C(C(=C1)Cl)Cl"  

# Convert the SMILES string to a molecule object
molecule = Chem.MolFromSmiles(canonical_monomer_smiles)

# Convert the molecule object back to canonical SMILES
canonical_smiles = Chem.MolToSmiles(molecule, canonical=True)

print("Canonical SMILES:", canonical_smiles)


Canonical SMILES: Clc1ccccc1Cl
